In [1]:
import sys, os
import tqdm
import numpy as np
import pandas as pd
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool
from torch_geometric.utils import degree

from chem import *

In [2]:
batch_size = 32
dataset_name = 'bbbp'

In [3]:
train_dataset = load_dataset('data_curation/train_set.csv')
val_dataset = load_dataset('data_curation/val_set.csv')
test_dataset = load_dataset('data_curation/test_set.csv')

In [4]:
train_dataset[0]

Data(x=[19, 25], edge_index=[2, 39], y=[1])

In [5]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [6]:
# Compute the maximum in-degree in the training data.
max_degree = -1
for data in train_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long)
for data in train_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import PNAConv

In [8]:
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        
        self.conv1 = PNAConv(25, 16, aggregators=aggregators, scalers=scalers, deg=deg)
        self.conv2 = PNAConv(16, 8, aggregators=aggregators, scalers=scalers, deg=deg)
        self.fc1 = torch.nn.Linear(8,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = global_mean_pool(x, data.batch)
        x = global_add_pool(x, data.batch)
        #x = global_max_pool(x, data.batch)
        x = self.fc1(x)

        return F.log_softmax(x, dim=1)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [10]:
model.train()
for epoch in range(2000):
    
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        loss_all += data.num_graphs * loss.item()
    if (epoch+1)%100 == 0:
        print (f"epoch {epoch+1}: ",loss_all / len(train_dataset))

epoch 100:  0.24089061632834644
epoch 200:  0.22132454930383832
epoch 300:  0.19492719345740347
epoch 400:  0.1912482905468277
epoch 500:  0.1833875470050707
epoch 600:  0.1776415389339519
epoch 700:  0.18408920218943087
epoch 800:  0.1781946941981792
epoch 900:  0.17955591788619374
epoch 1000:  0.16917037953817354
epoch 1100:  0.17498786810495456
epoch 1200:  0.17239623644248173
epoch 1300:  0.16062833431818088
epoch 1400:  0.16364653829706732
epoch 1500:  0.1635293838488661
epoch 1600:  0.15091514264894074
epoch 1700:  0.17140950127330606
epoch 1800:  0.15415203765007268
epoch 1900:  0.16320096110727508
epoch 2000:  0.15879495701382015


In [11]:
model.eval()
correct = 0
for data in train_loader:
    data = data.to(device)
    pred = model(data).max(dim=1)[1]
    correct += pred.eq(data.y).sum().item()
print (correct / len(train_dataset))

0.8982219497240956


In [12]:
model.eval()
correct = 0
for data in test_loader:
    data = data.to(device)
    pred = model(data).max(dim=1)[1]
    correct += pred.eq(data.y).sum().item()
print (correct / len(test_dataset))

0.5490196078431373


In [13]:
model.eval()
correct = 0
for data in val_loader:
    data = data.to(device)
    pred = model(data).max(dim=1)[1]
    correct += pred.eq(data.y).sum().item()
print (correct / len(val_dataset))

0.7352941176470589
